In [46]:
import os 
import numpy as np 
from typing import List, Callable, Tuple, Dict
import pandas as pd 


  
SEG_MAP_EMPTY = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
SEG_MAP = {
    'Proximal':[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    'Mid':[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    'Distal':[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
    'First marginal':[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
    'PDA':[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
    'First diagonal':[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
    'Ramus intermedius':[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
    'Third marginal':[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
    'Second marginal':[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
    'PLV':[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
    'Second diagonal':[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
}

ARTERY_MAP_EMPTY = [0,0,0]
ARTERY_MAP =  {
    'CX': [0,0,1],
    'LAD': [0,1,0],
    'RCA': [1,0,0],
    'RCA-pdaplv': [1,0,0],
    'LAD-d1': [0,0,1],
    'CX-om1': [0,0,1],
    'LAD-d2': [0,1,0]
}

LESION_TYPE_EMPTY = [0,0,0,0]

def artery_segment_to_float(seg: str) -> List[int]:
    if pd.isna(seg): return SEG_MAP_EMPTY
    elif seg in SEG_MAP:
        return SEG_MAP[seg]
    else:
        raise TypeError(f"Unknown segment string '{seg}'")

def artery_to_1hot(artery:str) -> List[int]:
    if pd.isna(artery): return ARTERY_MAP_EMPTY
    elif artery in ARTERY_MAP:
        return ARTERY_MAP[artery]
    else:
        raise TypeError(f"Unknown artery string '{artery}'")

def lesion_type_to_1hot(lesion_type:float) -> List[int]:
    # artery type is always a num between 1 and 4
    if pd.isna(lesion_type): return LESION_TYPE_EMPTY
    elif lesion_type <= 4.0:
        onehot = [0,0,0,0]
        onehot[int(lesion_type)-1] = 1
        return onehot
    else:
        raise TypeError(f"Unknown lesion type '{lesion_type}'")


In [47]:
# the feature list are paired with the feature transforms 

features = ["DS", "FFR", "Lesion_Length", "Lesion_Type", "artery", "exactSegmentLocation"]
feature_transforms = [None, None, None,lesion_type_to_1hot,artery_to_1hot,artery_segment_to_float]

CLINICAL_DATA_FEATURES = ["Age", "HTN", "Hchol", "DM_Overall", "Ren_Ins", "PVD", "CVA",
  "Prev_MI", "Prev_PCI", "Male", "CAD", "Smoker"]

for f in CLINICAL_DATA_FEATURES:
    if f not in features:
        features.append(f)
        feature_transforms.append(None)

print(features)
print(feature_transforms)




['DS', 'FFR', 'Lesion_Length', 'Lesion_Type', 'artery', 'exactSegmentLocation', 'Age', 'HTN', 'Hchol', 'DM_Overall', 'Ren_Ins', 'PVD', 'CVA', 'Prev_MI', 'Prev_PCI', 'Male', 'CAD', 'Smoker']
[None, None, None, <function lesion_type_to_1hot at 0x7fe39416ce50>, <function artery_to_1hot at 0x7fe39416ca60>, <function artery_segment_to_float at 0x7fe3941ab790>, None, None, None, None, None, None, None, None, None, None, None, None]


In [48]:

clinical_event_df_filepath='/home/sun/data/fame2_clinical_events_n_patient_data_2year.csv'
df = pd.read_csv(clinical_event_df_filepath)  

# print(df)

fame2_dump_dir='/home/sun/data/FAME2labelling/'

#you need to change the lesion_id_syntax for your cases
# here i use the 01-0002_2 as the example to extract those clinical information vector 

lesion_id_syntax='01-0002_2' #patient name_lesion type

filtered_data = df[df['lesion_id_syntax'] == lesion_id_syntax]
print(filtered_data)


  lesion_id_syntax  Patient  OMT_RegistryFU   FFR  FFR80  OneMinusFFR  MACE  \
0        01-0002_2  01-0002               3  0.83      0         0.17     0   

   VOCE  Days_MACE  Days_VOCE  ...  NUR_TVF  NUR_TVF_Days  MI_TVF  \
0     0        707        707  ...        0           709       0   

   MI_TVF_Days  CV_TVF  CV_TVF_Days  syntax  syntaxID  artery  \
0          709       0          709     3.0         2     RCA   

   exactSegmentLocation  
0                   Mid  

[1 rows x 60 columns]


In [49]:
def extract_lesion_wide(df_info, features, transforms):
    lesion_wide_features, names = [], []

    for i in range(len(features)):
        
        feature = features[i]
        transform = transforms[i]
        
        if transform != None:

            # value = transform( getattr(df_info,feature))
            value = transform(df_info[feature].tolist()[0])

            if isinstance(value, list):
                lesion_wide_features.extend(value)
                names.extend( [features[i]] * len(value) )
            else:
                lesion_wide_features.append(value)
        else:
            value = df_info[feature].tolist()[0]
            if pd.isna(value):
                value = 0.0
            lesion_wide_features.append(value)
            names.append(features[i])

    return lesion_wide_features, names


lesion_wide_features, names = extract_lesion_wide(filtered_data,features,feature_transforms)
# print(filtered_data[features[4]].tolist())
print(lesion_wide_features)

[63.48, 0.83, 39.8, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 65, 1, 1.0, 0, 0, 1, 0, 1.0, 1.0, 0, 1, 0]
